# Matching SkySim5000 DM halo
Authors : Michel Aguena

Matching clusters using the [ClEvaR](https://lsstdesc.org/clevar/) DESC library.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import GCRCatalogs

%matplotlib inline

In [ ]:
skysim_cat = GCRCatalogs.load_catalog('skysim5000_v1.1.1_small')

In [ ]:
cosmo_ss  = skysim_cat.cosmology

In [ ]:
cosmo_ss

## Extract DM haloes from the catalog in a given mass and redshift range. 

In [ ]:
%%time
# get list of halos in a given redshift and mass range
dm_halos = Table(skysim_cat.get_quantities(
    ['halo_id', 'halo_mass', 'redshift','ra', 'dec',
     'baseDC2/sod_halo_mass','baseDC2/sod_halo_radius'],
    filters=[
        f'halo_mass > {10**13.95}',
        'is_central==True',
        f'redshift>{0}',
        f'redshift<{1}']
))
dm_halos['m200c'] = dm_halos['baseDC2/sod_halo_mass']/cosmo_ss.h

In [ ]:
%%time
# get list of halos in a given redshift and mass range
gals = Table(skysim_cat.get_quantities(
    ['galaxy_id', 'halo_id', 'halo_mass', 'redshift','ra', 'dec'],
    filters=[
        f'halo_mass > {10**13.95}',
        f'redshift>{0}',
        f'redshift<{1.1}']
))

In [ ]:
print(f'There are {len(dm_halos):,} halos in this mass (Mfof) and redshift range.')
print(f'They contain {len(gals):,} galaxies in total.')

In [ ]:
plt.scatter(gals['ra'], gals['dec'], marker='.', s=.1)
plt.scatter(dm_halos['ra'], dm_halos['dec'], marker='.', s=10)
plt.xlabel('ra [deg]')
plt.ylabel('dec [deg]')

### Define 2 halo catalogs
- halos1: $M_{200c}>10^{14}M_\odot$
- halos2: $M_{fof}>10^{14}M_\odot$

In [ ]:
plt.scatter(dm_halos['m200c'], dm_halos['halo_mass'], marker='.', s=10)
plt.xlabel('$M_{200c}$ [$M_\odot$]')
plt.ylabel('FoF mass [$M_\odot$]')
diag = dm_halos['halo_mass'].min(), np.max([*dm_halos['m200c'], *dm_halos['halo_mass']])
plt.plot(diag, diag, c='r', ls='--')
plt.axvline(1e14, c='g')
plt.axhline(1e14, c='y')
plt.xscale('log')
plt.yscale('log')

Import clevar and create catalogs

In [ ]:
import clevar
from clevar import ClCatalog

In [ ]:
dm_halos1 = dm_halos[dm_halos['m200c']>1e14]
halos1 = ClCatalog(
    name='M200',
    id=dm_halos1['halo_id'],
    ra=dm_halos1['ra'],
    dec=dm_halos1['dec'],
    z=dm_halos1['redshift'],
    mass=dm_halos1['m200c'],
    radius=dm_halos1['baseDC2/sod_halo_radius']                   
)
halos1.radius_unit = 'mpc'
del dm_halos1

# Nice view formatting
halos1['mass'].info.format = '.3g'
halos1['ra'].info.format = '.4f'
halos1['dec'].info.format = '.4f'
halos1['z'].info.format = '.4f'
halos1['radius'].info.format = '.2f'

halos1.labels['mass'] = 'M_{200c}'

In [ ]:
dm_halos2 = dm_halos[dm_halos['halo_mass']>1e14]
halos2 = ClCatalog(
    name='Mfof',
    id=dm_halos2['halo_id'],
    ra=dm_halos2['ra'],
    dec=dm_halos2['dec'],
    z=dm_halos2['redshift'],
    mass=dm_halos2['halo_mass'],
    radius=dm_halos2['baseDC2/sod_halo_radius']                         
)
halos2.radius_unit = 'mpc'
del dm_halos2

# Nice view formatting
halos2['mass'].info.format = '.3g'
halos2['ra'].info.format = '.4f'
halos2['dec'].info.format = '.4f'
halos2['z'].info.format = '.4f'
halos2['radius'].info.format = '.2f'

halos2.labels['mass'] = 'M_{fof}'

In [ ]:
display(halos1[:5])
display(halos2[:5])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

mbins = np.logspace(14, 15, 21)
axes[0].hist(halos1['mass'], bins=mbins, label='halos1 ($M_{200c}$)')
axes[0].hist(halos2['mass'], bins=mbins, label='halos2 ($M_{fof}$)', histtype='step')
axes[0].set_xscale('log')
axes[0].set_xlabel('Mass')
axes[0].legend()

zbins = np.linspace(0, 1, 21)
axes[1].hist(halos1['z'], bins=zbins)
axes[1].hist(halos2['z'], bins=zbins, histtype='step')
axes[1].set_xlabel('redshift')


Add members:

In [ ]:
halos1.add_members(
    id=gals['galaxy_id'],
    id_cluster=gals['halo_id'])

In [ ]:
halos2.add_members(
    id=gals['galaxy_id'],
    id_cluster=gals['halo_id'])

In [ ]:
display(halos1.members[:5])
display(halos2.members[:5])

## Match catalogs

By proximity:

In [ ]:
from clevar.match import ProximityMatch
from clevar.cosmology import AstroPyCosmology
mtp = ProximityMatch()
cosmo = AstroPyCosmology()

print("\n* prepare the input for matching")
print("--------------------------------")
mtp.prep_cat_for_match(halos1, delta_z=0.01, match_radius='1 arcsec')
mtp.prep_cat_for_match(halos2, delta_z=0.01, match_radius='cat', cosmo=cosmo)

print('\n* find multiple matches')
print("-----------------------")
mtp.multiple(halos1, halos2)
mtp.multiple(halos2, halos1)


print('\n* find unique pairs')
print("-------------------")
mtp.unique(halos1, halos2, 'more_massive')
mtp.unique(halos2, halos1, 'more_massive')

print('\n* cross matches')
halos1.cross_match()
halos2.cross_match()

# Nice formatting
halos1.mt_input['ang'].info.format = '.6f'
halos2.mt_input['ang'].info.format = '.6f'

In [ ]:
display(halos1[:5])
display(halos2[:5])

By members:

In [ ]:
halos1._init_match_vals(True)
halos2._init_match_vals(True)

In [ ]:
from clevar.match import MembershipMatch
mtm = MembershipMatch()

print("\n* prepare the input for matching")
print("--------------------------------")
print("\n    * start by matching the members")
mtm.match_members(halos1.members, halos2.members, method='id')

print('\n    * fill corresponding share members')
mtm.fill_shared_members(halos1, halos2)

print('\n* find multiple matches')
print("-----------------------")
mtm.multiple(halos1, halos2)
mtm.multiple(halos2, halos1)


print('\n* find unique pairs')
print("-------------------")
mtm.unique(halos1, halos2, 'shared_member_fraction')
mtm.unique(halos2, halos1, 'shared_member_fraction')

print('\n* cross matches')
halos1.cross_match()
halos2.cross_match()

# Nice formatting
halos1['mt_frac_self'].info.format = '.2f'
halos1['mt_frac_other'].info.format = '.2f'
halos2['mt_frac_self'].info.format = '.2f'
halos2['mt_frac_other'].info.format = '.2f'

In [ ]:
display(halos1[:5])
display(halos2[:5])

## Metrics of matching

### Recovery rate

In [ ]:
from clevar.match_metrics import recovery

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

mbins = np.logspace(14, 15, 4)
zbins = np.linspace(0, 1, 11)
cp_info1 = recovery.plot(
    halos1, 'multi_join', zbins, mbins,
    shape='line', ax=axes[0])
cp_info2 = recovery.plot(
    halos2, 'multi_join', zbins, mbins,
    shape='line', ax=axes[1])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

mbins = np.logspace(14, 15, 21)
zbins = [0, .5, 1]
cp_info1 = recovery.plot(
    halos1, 'multi_join', zbins, mbins,
    shape='line', ax=axes[0], transpose=True,
)
cp_info2 = recovery.plot(
    halos2, 'multi_join', zbins, mbins,
    shape='line', ax=axes[1], transpose=True,
)

### Scaling relation

In [ ]:
from clevar.match_metrics import scaling

In [ ]:
info_mr1 = scaling.redshift_density_metrics(
    halos1, halos2, 'cross', ax_rotation=45)

In [ ]:
info_mr1 = scaling.mass_density_metrics(
    halos1, halos2, 'cross', ax_rotation=45)

In [ ]:
info_mr1 = scaling.mass_density_metrics(
    halos1, halos2, 'cross', ax_rotation=45,
    add_fit=True, fit_log=True, fit_bins1=5,
)